In [1]:
from torch import nn
import torch.nn.functional as F
import torch
from fight_env import fight_env
from fight_train_func import rewardFunction, normalizeData
import random
from fightAlgorithm import testDataForAgent
from PPO import PPO



class PolicyNetwork(nn.Module):
    def __init__(self, state_size, action_size):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, action_size)

        

    def forward(self, x):
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        logits = self.fc3(x)
        return F.log_softmax(logits, dim=-1)

        

        
class ValueNetwork(nn.Module):
    def __init__(self, state_size):
        super(ValueNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)  # 僅一個輸出，表示狀態的價值
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    

class MyPPO(PPO):
    def __init__(self, env, policyNetwork, valueNetwork):
        super().__init__(policyNetwork, valueNetwork)
        self.env = env

    def show(self):
        device = torch.device("cpu")
        self.PolicyNetwork.to(device, dtype=torch.float32)
        self.ValueNetwork.to(device, dtype=torch.float32)
        data = self.env.reset()
        while (True):    
            self.env.render()                
            updateData = {}
            
            for j in range(1, 7):
                playerID = str(j) + "P"

                state, _, _ = data[playerID]
                action = self.getAction(normalizeData(state))    
                updateData[playerID] = [self.env.actionSpace[action]]
            data = self.env.update(updateData)  


            if (not self.env.not_done()):
                break

    def learn(self, timeStep=10000, dataNum = 4096, lr=0.003, episode=0.2, epoch=10, batchSize=256):
        print("start learning")
        for i in range(timeStep):
            playtime_count = 0
            
            
            
            while (len(self.ExperienceHistory['oldstate']) < dataNum):
                data = self.env.reset()
                die = [False, False, False, False, False, False, False]
                agentRewards = [0, 0, 0, 0, 0, 0, 0]
                while (True):                
                    updateData = {}
                    oldStates = {}
                    actions = {}
                    end_game = not self.env.not_done()
                    for j in range(1, 7):
                        playerID = str(j) + "P"
                        if (self.env.lives[j] == 0):
                            die[j] = True
                            continue
                        
                        state, _, _ = data[playerID]
                        oldStates[playerID] = state
                        action = self.getAction(normalizeData(state))
                       
                        
                        updateData[playerID] = [self.env.actionSpace[action]]
                        actions[playerID] = action
                    data = self.env.update(updateData)  

                    for j in range(1, 7):
                        if (not die[j]):
                            playerID = str(j) + "P"
                            old_state = oldStates[playerID]
                            new_state, liveLoss, scoreUp = data[playerID]
                            action = actions[playerID]
                            reward = rewardFunction(old_state, self.env.actionSpace[action], scoreUp, liveLoss)
                            
                            done = int(self.env.lives[j] == 0 or end_game)
                            
                            self.ExperienceHistory['oldstate'].append(normalizeData(old_state))
                            self.ExperienceHistory['state'].append(normalizeData(new_state))
                            self.ExperienceHistory['action'].append(action)
                            self.ExperienceHistory['reward'].append(reward)
                            self.ExperienceHistory['done'].append(done)

                            agentRewards[j] += reward

                                

                    if (end_game):
                        if (i % 10 == 0):
                            print(f"time step:{i + 1}", end=" ")
                            for j in range(1, 7):
                                
                                print(f"player{j} reward: {agentRewards[j]}", end=",")
                            print()
                        playtime_count += 1
                        break
            self.train(epochs=epoch, lr=lr, episode=episode, batch_size=batchSize)





pygame 2.5.2 (SDL 2.28.3, Python 3.9.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
env = fight_env(FPS=300)
policyNetwork = PolicyNetwork(11, 7)
valueNetwork = ValueNetwork(11)
agent = MyPPO(env, policyNetwork=policyNetwork, valueNetwork=valueNetwork)


In [3]:
agent.learn(timeStep=10000, lr=0.003, dataNum=4096)

start learning
time step:1 player1 reward: 6.599999999999973,player2 reward: -8.58999999999999,player3 reward: -6.5899999999999945,player4 reward: 15.479999999999936,player5 reward: -2.2799999999999954,player6 reward: -5.019999999999997,
time step:1 player1 reward: 4.589999999999904,player2 reward: -1.4899999999999967,player3 reward: -8.219999999999997,player4 reward: 20.60000000000034,player5 reward: -7.169999999999984,player6 reward: -4.759999999999943,


c:\Users\weiso131\Desktop\gameAI\weiso_tank_env\PPO.py:146: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  with torch.autograd.detect_anomaly():


time step:11 player1 reward: 2.6899999999999524,player2 reward: -6.539999999999998,player3 reward: -4.309999999999949,player4 reward: 9.02999999999995,player5 reward: -5.6699999999999235,player6 reward: -8.269999999999872,
time step:11 player1 reward: 14.279999999999953,player2 reward: -7.1999999999999895,player3 reward: -11.429999999999959,player4 reward: 17.01999999999991,player5 reward: -8.289999999999967,player6 reward: -6.169999999999998,


In [5]:
import pygame

for i in range(10):
    agent.show()
    pygame.quit()